In [29]:
import math 
import numpy as np
from numpy.typing import NDArray
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import random as rng
import networkx as nx
import matplotlib.pyplot as plt
import pickle
from typing import Union, List, Any
from qaoa_and_tdvp import QAOA, tdvp_optimize_qaoa, scipy_optimize, QAOAResult, Qobj
from MaxCut import MaxCut
from itertools import product

from qutip import tensor, basis, Qobj, expect, variance
from qutip.operators import qeye, sigmax, sigmay, sigmaz
from qutip.qip.operations import expand_operator, rz

import swifter

from benchmark import (
    get_all_connected,
    get_connected_rn_graph,
    get_rn_qubo,
    select_if_connected,
    Benchmark,
)
from qutip.parallel import parallel_map, serial_map

# convert data into DataFrame

In [30]:
with open('.//results//n5_results.p', 'rb') as file:
    df = pickle.load(file)

In [2]:
# with open('.//results//n4_p1.p','rb') as file:
#     df = pickle.load(file)

In [ ]:
# with open("..//benchmarks//RK45_n10_p-1-5_Delta-1.p","rb") as file:
#     data: Benchmark = pickle.load(file)
#
# scipy_results = list()
# for d in data.results:
#     items = d.get("tdvp").__dict__.items()
#     for k,v in items:
#         d[k]=v
#         d["algorithm"] = "tdvp"
#     d["n"] = d["_qaoa"].n
#     dd = d.copy()
#     items = dd.get("scipy").__dict__.items()
#     for k,v in items:
#         dd[k]=v
#         dd["algorithm"] = "scipy"
#     dd["n"] = dd["_qaoa"].n
#     scipy_results.append(dd)
#
# data.results.extend(scipy_results)
#
# df = pd.DataFrame(data.results)
# df.pop("tdvp")
# df.pop("scipy")
# cols = df.columns.tolist()
# cols = cols[-1:] + cols[:-1]
# df = df[cols]
#
# df['steps'] = df.apply(lambda row: int(row['_num_steps']) if row.algorithm=="tdvp" else row['_num_fun_calls'], axis=1)
#
# df.insert(2,"algorithm",df.pop("algorithm"))
# df.insert(3,"_optimal_fun_value",df.pop("_optimal_fun_value"))
# df.insert(4,'steps', df.pop('steps'))
# df.insert(6,"_prob",df.pop("_prob"))
#
#
# df_tdvp = df[df.algorithm.eq("tdvp")]
# df_scipy = df[df.algorithm.eq("scipy")]
#
# df.rename(columns={"_optimal_fun_value":"C", "_prob":"prob", '_duration':'duration', '_optimal_state':'final_state', '_optimal_parameters':'final_parameters', '_num_gates':'num_gates','_qaoa':'qaoa'}, inplace=True)

In [56]:
er:pd.Series = df['tdvp'].loc[3,2]

In [59]:
for i in er.index:
    try:
        er[i].__repr__()
    except TypeError:
        print(i)

res


In [62]:
res :QAOAResult= er.res

In [41]:
df

TypeError: unsupported format string passed to NoneType.__format__

TypeError: unsupported format string passed to NoneType.__format__

# Compute performance measures

In [47]:
for (p,i) in df.index:
    try:
        df['tdvp'].loc[p,i].__repr__()
    except TypeError:
        print((p,i))
        continue

(3, 2)


In [28]:

dfp = df[df['p']==1]
measures = ['expected approximation ratio', 'groundspace overlap', 'groundspace sharpness', 'Approximation ratio', 'circuit count', 'gate count']
types = ['min','max', 'mean', 'stdv']
measure_index=pd.MultiIndex.from_product([measures, types])
summary = pd.DataFrame(columns=measure_index)
summary

Empty DataFrame
Columns: [(expected approximation ratio, min), (expected approximation ratio, max), (expected approximation ratio, mean), (expected approximation ratio, stdv), (groundspace overlap, min), (groundspace overlap, max), (groundspace overlap, mean), (groundspace overlap, stdv), (groundspace sharpness, min), (groundspace sharpness, max), (groundspace sharpness, mean), (groundspace sharpness, stdv), (Approximation ratio, min), (Approximation ratio, max), (Approximation ratio, mean), (Approximation ratio, stdv), (circuit count, min), (circuit count, max), (circuit count, mean), (circuit count, stdv), (gate count, min), (gate count, max), (gate count, mean), (gate count, stdv)]
Index: []

[0 rows x 24 columns]

In [3]:
def prob(num:int, state:Qobj):
    n = state.shape[0]
    return abs(state.overlap(basis(n,num)))**2

def answer(state:Qobj):
    return max(range(state.shape[0]), key=lambda x:prob(x,state))

def bin_answer(state:Qobj):
    return format(answer(state),  f'0{len(state.dims[0])}b')

In [4]:
def best_cost_value(qaoa:QAOA):
    return qaoa.H.groundstate()[0]

def approximation_ratio(bin_answer:int, best_cost_value:float, qaoa:QAOA):
    i = list(int(b) for b in bin_answer)
    dims = [2 for _ in range(qaoa.n)]
    return expect(qaoa.H,basis(dims,i))/best_cost_value

def expectation_of_approximation_ratio(state:Qobj, best_cost_value:float, qaoa:QAOA):
    return (expect(qaoa.H, state))/best_cost_value


In [22]:
def num_metric_gates(n:int,p:int,qubo:NDArray):
    Q = np.count_nonzero(qubo)
    N = 2*p*(n+Q)
    C1 = 1/2 * n**4 + 2*n**3 + 7/2*n**2
    C2 = n**4 + 5/2*(n**3 + n**2)
    return N*C1*(7/3*p**2 +2*p + 1/3) + (2*C2-N*C1)*(1/2*p**2 +3/2*p)

def num_gradient_gates(n:int,p:int,qubo:NDArray):
    Q = np.count_nonzero(qubo)
    N = 2*p*(n+Q)
    return (n**4 -2*n**3 +n**2)/4 * (2*N+4) + 3*n*(n*(n-1))/2*(2*(N+3)) + 2*n**2*(2*N+3)

def num_qaoa_gates(n:int,p:int, qubo:NDArray):
    Q = np.count_nonzero(qubo)
    return 2*p*(n+Q)

def circuit_count(steps:int, algorithm:str, p:int, n:int):
    if algorithm =='tdvp':
        return steps*(p**2 + p)*(1/8*n**4 +1/2*n**3 -5/8*n**2) + 1
    if algorithm =="scipy":
        return steps + 1

def gate_count(steps:int, algorithm:str, qaoa:QAOA, p:int, n:int):
    if algorithm=='tdvp':
        return steps*(num_metric_gates(n,p,qaoa.qubo) + num_gradient_gates(n,p,qaoa.qubo)) + num_qaoa_gates(n,p,qaoa.qubo)
    if algorithm=='scipy':
        return (1+steps)*num_qaoa_gates(n,p,qaoa.qubo)

In [6]:
def groundspace(qaoa:QAOA):
    eigenenergies, eigenstates = qaoa.H.eigenstates(sort='low')
    groundenergy = eigenenergies[0]

    num_of_eigenstates = 0
    for e in eigenenergies:
        if e==groundenergy:
            num_of_eigenstates +=1
        if e != groundenergy:
            break
    return eigenstates[:num_of_eigenstates]

def groundspace_overlap(groundstates:List[Qobj], state:Qobj):
    return  sum(abs(g.overlap(state))**2 for g in groundstates)

def groundspace_sharpness(groundstates:List[Qobj], state:Qobj):
    P_0 = sum(g*g.dag() for g in groundstates)
    return math.sqrt(variance(P_0,state))


In [10]:
# df['groundspace'] = df.swifter.apply(lambda row: groundspace(row.qaoa), axis=1)

In [11]:
# df.insert(3,'groundspace_overlap', df.swifter.apply(lambda row: groundspace_overlap(row['groundspace'], row['final_state']), axis=1))

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

ValueError: cannot insert groundspace_overlap, already exists

In [12]:
# df.insert(4, 'groundspace_sharpness', df.swifter.apply(lambda row: groundspace_sharpness(row['groundspace'], row['final_state']), axis=1))

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

ValueError: cannot insert groundspace_sharpness, already exists

In [14]:
df['steps']

0        24
1        30
2        37
3        44
4        31
       ... 
995      98
996     469
997     441
998     782
999    1000
Name: steps, Length: 1000, dtype: int64

In [15]:
df['circuit_count']= df.swifter.apply(lambda row: circuit_count(row['steps'], row['algorithm'], row['p'], row['n']), axis=1)

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

In [23]:
df['gate_count'] =df.swifter.apply(lambda row: gate_count(row['steps'], row['algorithm'],row['qaoa'], row['p'], row['n']), axis=1)

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

In [19]:
df['answer'] = df.swifter.apply(lambda row: answer(row['final_state']), axis=1)

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

In [51]:
df['bin_answer'] = df.swifter.apply(lambda row: bin_answer(row['final_state']), axis=1)

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

In [52]:
df.insert(3, 'best_cost_value', df.swifter.apply(lambda row: best_cost_value(row['qaoa']), axis=1))

KeyboardInterrupt: 

In [53]:
df.insert(3, 'approximation_ratio', df.swifter.apply(lambda row: approximation_ratio(row['bin_answer'],row['best_cost_value'],row['qaoa']), axis=1))

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

In [56]:
df.insert(3, 'expectation_of_approximation_ratio', df.swifter.apply(lambda row: expectation_of_approximation_ratio(row['final_state'],row['best_cost_value'],row['qaoa']), axis=1))

Pandas Apply:   0%|          | 0/1000 [00:00<?, ?it/s]

In [25]:
# with open('..//benchmarks//n10_data.p', 'wb') as file:
#     pickle.dump(df,file)